# WH_4 - Web scraping

## importing ibraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

## scraping function

In [3]:
def scrape_madlan(city): #Between the words, use a hyphen. For example ('כפר-סבא')
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
    url = "https://www.madlan.co.il/for-sale/" + city + "-ישראל"
    response = requests.get(url,headers= headers)
    
    try:
        if not response.status_code == 200:
            print("You have a problem with the request for madlan's home page - status_code:", response.status_code)
            return None
    
        results_page_assets = BeautifulSoup(response.content, 'html.parser')

        # An empty list to store asset details
        assets_results = list()

        # Find all assets on the page
        assets_lst = results_page_assets.find_all('div',{'class':"css-zuyt5t e1sx3tzs1",'data-auto': "listed-bulletin",'data-auto-bulletin-type':"bulletin"})

        city = results_page_assets.find('div' ,{'class':'css-7so23d e3vrfmg6','data-auto':"breadcrumb-text"}).text
        # loop over each asset and extract the details
        for asset in assets_lst:
            asset_details = {}
            asset_property_class = asset.find('div', {'data-auto': 'property-class'})
            asset_details['City'] = city
            asset_details['type'] = asset_property_class.get_text() if asset_property_class is not None else None

            asset_room_number = asset.find("div", {"data-auto": "property-rooms"})
            asset_room_number_string = asset_room_number.text.strip() if asset_room_number else None
            asset_details['room_number'] = str(re.sub(r'[^\d.-]', '', asset_room_number_string) if asset_room_number_string else None)

            asset_area = asset.find("div", {"data-auto": "property-size"})
            asset_area_string = asset_area.text.strip() if asset_area else "None"
            asset_area_numbers_only = re.findall(r'\d+', asset_area_string)
            asset_details['Area'] = int(asset_area_numbers_only[0]) if asset_area_numbers_only else None

            asset_address = asset.find('div', {'data-auto': 'property-address'})
            asset_address = asset_address.get_text(strip=True) if asset_address else "None"
            Street_and_num = asset_address.split(',')[0] if asset_address != "None" else "None"
            Street_and_num = Street_and_num.split()
            Street = ''
            num = ''
            for part in Street_and_num:
                if part.isnumeric():
                    num = num + part
                else:
                    Street = Street + ' ' + part
            asset_details['Street'] = str(Street.strip()) if Street != '' else None
            asset_details['number_in_street'] = float(num) if num != '' else None
            asset_details['city_area'] = str(asset_address.split(',')[1].strip() if len(asset_address.split(',')) > 1 else None)

            price_element = asset.find("div", {"data-auto": "property-price"})
            price_string = price_element.text.strip() if price_element else "None"
            num_match = re.search(r'\d+(?:,\d+)*', price_string)
            asset_details['price'] = int(num_match.group().replace(',', '')) if num_match else None

            ############################# Bonus #################################################### 

            asset_url = "https://www.madlan.co.il" + asset.find('a').get('href')
            response_asset = requests.get(asset_url,headers= headers)
            results_page_current_asset = BeautifulSoup(response_asset.content,'html.parser')
            photos = results_page_current_asset.find_all('div',{'class': 'css-18nlt6 e1r3dysu3'})
            asset_details['num_of_images'] = len(photos)
            asset_details['floor_out_of'] = results_page_current_asset.find('div', {'data-auto': 'floor'}).get_text() if results_page_current_asset.find('div', {'data-auto': 'floor'}) is not None else None
            blocks_lst = results_page_current_asset.find_all('div',{'class':'css-1wpv10e e125ttrt3'})
            title_names = ['hasElevator','hasAirCondition','hasParking','hasBalcony','hasBars','hasMamad','hasStorage','handicapFriendly']
            for block in range (len(blocks_lst)-1):
                asset_details[title_names[block]] = blocks_lst[block].get_text()

            asset_condition = results_page_current_asset.find("div", {"data-auto": "unit-general-condition-value"})
            asset_details['condition'] = str(asset_condition.get_text() if asset_condition  is not None else None)

            asset_entranceDate = results_page_current_asset.find("div", {"data-auto": "unit-availability-value"})
            asset_details['entranceDate'] = str(asset_entranceDate.text.strip() if asset_entranceDate  is not None else None)

            asset_furniture = results_page_current_asset.find("div", {"data-auto": "unit-furniture-details-value"})
            asset_details['furniture'] = str(asset_furniture.text.strip() if asset_furniture  is not None else None)

            asset_details['publishedDays'] = None # Change in the assignment according to the lecturer to leave the column empty unless it is filled in manually

            asset_description = results_page_current_asset.find("div", {"class": "css-v1qjdi ebqee3y2"})
            asset_details['description'] = str(asset_description.text.strip() if asset_description  is not None else None)

            ############################# Bonus #################################################### 

            # Append asset details to the assets_results
            assets_results.append(asset_details)
        
    except:
        pass
    
    return assets_results


In [4]:
assets_details = pd.DataFrame(scrape_madlan('כפר-סבא'))

## The arrangement of the data frame

In [5]:
assets_details_save_copy = assets_details # Just in case
order_columns = ['City','type', 'room_number','Area', 'Street', 'number_in_street', 'city_area', 'price', 'num_of_images', 'floor_out_of',
                 'hasElevator', 'hasParking', 'hasBars', 'hasStorage', 'condition', 'hasAirCondition', 'hasBalcony', 'hasMamad',
                 'handicapFriendly','entranceDate', 'furniture','publishedDays', 'description']
assets_details = assets_details[order_columns]
columns_to_fillna = ['hasElevator', 'hasParking', 'hasBars', 'hasStorage', 'hasAirCondition', 'hasBalcony', 'hasMamad',
                 'handicapFriendly']
assets_details[columns_to_fillna] = assets_details[columns_to_fillna].fillna('False')

In [6]:
assets_details

,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,floor_out_of,...,hasStorage,condition,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description
0,כפר סבא,דירה,4,122.0,ביאליק,22.0,מרכז העיר דרום,3100000,11,קומה 1 מתוך 4,...,אין מחסן,שמור,יש מיזוג אויר,יש מרפסת,יש ממ״ד,לא נגיש לנכים,12/2023,חלקי,None,"דירת 4 חדרים, 122 מ״ר לפי ארנונה, קומה ראשונה ..."
1,כפר סבא,דירה,3,85.0,"הגר""א",6.0,מרכז העיר דרום,2850000,6,קומה 2 מתוך 3,...,אין מחסן,משופץ,יש מיזוג אויר,אין מרפסת,אין ממ״ד,לא נגיש לנכים,08/2023,מלא,None,"דירת 3 חדרים. מיקום מעולה, שקט ומרכזי, קרוב לת..."
2,כפר סבא,דירה,3.5,NaN,נחשון,NaN,מרכז העיר דרום,2300000,2,קומה 1 מתוך 5,...,אין מחסן,שמור,יש מיזוג אויר,אין מרפסת,אין ממ״ד,נגיש לנכים,גמיש,לא צויין,None,None
3,כפר סבא,דופלקס,5.5,NaN,ארלוזורוב,NaN,מרכז העיר דרום,4100000,1,קומה 5 מתוך 6,...,אין מחסן,ישן,יש מיזוג אויר,יש מרפסת,אין ממ״ד,לא נגיש לנכים,גמיש,לא צויין,None,למכירה במרכז העיר קרוב להכל דופלקס מעולה שייך ...
4,כפר סבא,דירה,4,125.0,ויצמן,78.0,מרכז העיר דרום,777777,5,קומה 3,...,אין מחסן,משופץ,יש מיזוג אויר,אין מרפסת,אין ממ״ד,נגיש לנכים,מיידי,לא צויין,None,לפני פינוי בינוי עתידי .
5,כפר סבא,דירה,5,132.0,נלקין,2.0,השכונה הירוקה,4200000,0,קומת קרקע,...,יש מחסן,חדש,יש מיזוג אויר,אין מרפסת,יש ממ״ד,נגיש לנכים,לא צויין,לא צויין,None,דירה 5 חדרים צופה על גינת נלקין המרכזית בשכונה...
6,כפר סבא,דירה,3,NaN,מוצקין/דובדבן הכפר/כיסופים,NaN,None,2400000,1,קומה 2 מתוך 4,...,אין מחסן,חדש,יש מיזוג אויר,יש מרפסת,יש ממ״ד,לא נגיש לנכים,לא צויין,לא צויין,None,"מציאה של ממש דירה עם חניה, ממד \nברחוב מבוקש ב..."
7,כפר סבא,דירה,3.5,84.0,יבנה,NaN,מוצקין/דובדבן הכפר/כיסופים,2490000,7,קומה 4 מתוך 6,...,אין מחסן,חדש,יש מיזוג אויר,יש מרפסת,יש ממ״ד,נגיש לנכים,גמיש,לא צויין,None,"דירה חדשה בבניין שעבר תמ""א 38"
8,כפר סבא,דירה,4,110.0,הרצל,26.0,מרכז העיר דרום,2890000,6,קומה 2 מתוך 5,...,אין מחסן,משופץ,יש מיזוג אויר,אין מרפסת,אין ממ״ד,לא נגיש לנכים,גמיש,לא צויין,None,"ללא תיווך דירה מעוצבת ושמורה מאוד, מזגן בכל חד..."
9,כפר סבא,דירה,5,143.0,רוטשילד,16.0,מרכז העיר דרום,3450000,13,קומה 2 מתוך 4,...,אין מחסן,משופץ,יש מיזוג אויר,אין מרפסת,אין ממ״ד,לא נגיש לנכים,10/2023,מלא,None,https://dira.internetido.com/\nלמכירה (ללא תיו...


## Export to CSV

In [13]:
assets_details.to_csv('C:\\Users\\Student\\assets_details.csv', encoding='utf-8-sig', index=False)